In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Read in dataset from (1)
df = pd.read_csv('Data.csv')

In [ ]:
# Change column names
df.rename(columns = 
    {'datadate': 'Date', 'conm_x': 'Name', 'sedol_x': 'ID', 'curcdq': 'Currency', 'prccd': 'Price',
    'atq': 'Total Assets', 'apq': 'Accounts Payable', 'ltq': 'Total Liabilities', 'niq': 'Net Income',
    'oiadpq': 'EBIT', 'revtq': 'Revenue', 'dpq': 'Depreciation', 'gpq': 'Gross Profit',
    'invtq': 'Inventories', 'rectq': 'Receivables', 'saleq': 'Sales/Turnover', 'teqq': 'Stockholders Equity',
    'xintq': 'Interest Expense', 'ebitda': 'EBITDA', 'fcfq': 'Free Cash Flow', 'fincfq': 'Financing Cash Flow',
    'ivncfq': 'Investing Cash Flow', 'oancfq': 'Operating Cash Flow', 'lctq': 'Current Liabilities', 'actq': 'Current Assets',
    'cogsq': 'Cost of Goods Sold', 'roa': 'ROA', 'roe': 'ROE', 'roi': 'ROI',
    'gpm': 'Gross Profit Margin', 'opm': 'Operating Profit Margin', 'npm': 'Net Profit Margin', 'cr': 'Current Ratio',
    'qr': 'Quick Ratio', 'de': 'D/E Ratio', 'icr': 'Interest Coverage Ratio', 'atr': 'Asset Turnover Ratio',
    'itr': 'Inventory Turnover Ratio', 'og': 'Operational Gearing', 'rg': 'Revenue Growth', 'eag': 'Earnings Growth',
    'ag': 'Asset Growth', 'eqg': 'Equity Growth', 'acr': 'Accruals Ratio', 'cftnir': 'Cashflow to Net Income Ratio',
    'prchd': 'High Price', 'prcld': 'Low Price', 'cshtrd': 'Volume', 'cshoc': 'Shares', 'return' : 'Return'}, inplace = True)

In [ ]:
# Drop rows with the same name on the same date, keeping the first 
df = df.drop_duplicates(['Name','Date'],keep= 'first')

In [ ]:
# Set date to datetime type
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
# Filter start and end date to remove NaNs
start_date = pd.to_datetime('2010-03-31')
end_date = pd.to_datetime('2023-12-31')

df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

In [ ]:
# Drop rows where certain columns are zero
df = df[df['Revenue']!=0]
df = df[df['Total Assets']!=0]
df = df[df['Total Liabilities']!=0]
df = df[df['Price']!=0]
df = df[df['Shares']!=0]

In [ ]:
# Calculate sharehlders equity
df['Equity'] = df['Total Assets'] - df['Total Liabilities']

In [ ]:
# Calculate market cap
df['Market Cap'] = df['Price']*df['Shares']

## Cap Returns to remove outliers

In [ ]:
# Set date to datetime type
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
# Get unique list of dates
dates1 = pd.to_datetime(df['Date'].unique().tolist())

In [ ]:
# Find the 5th and 95th percentile of the returns at each quarter
max_key = {}
min_key = {}

for d in dates1:
    temp = df[df['Date']==d]
    if len(temp['Return'].dropna())>0:
        mx = np.percentile(temp['Return'].dropna(),95)
        mn = np.percentile(temp['Return'].dropna(),5)
    else:
        mx = mn = np.nan
    max_key[d] = mx
    min_key[d] = mn

In [ ]:
# Constrain returns between 5% and 95%
for i in range(len(df)):
    
    date = df.iloc[i,0]

    mx = max_key[date]
    mn = min_key[date]
    if df.iloc[i,5] > mx:
        df.iloc[i,5] = mx
    if df.iloc[i,5] < mn:
        df.iloc[i,5] = mn

## Other Variables

In [ ]:
# Drop rows with the same name on the same date, keeping the first 
df = df.drop_duplicates(['Name','Date'],keep= 'first')

In [ ]:
# Get mean return of market at each period. I.E. the market return
market_ret = df.groupby('Date')['Return'].mean()

In [ ]:
# Set index as date
df.set_index('Date', inplace=True, drop=False)

In [ ]:
# Join main data with the market mean return at each period
df = df.join(market_ret, how='left' , lsuffix='', rsuffix='_Market')

In [ ]:
# Calculate alpha
df['Alpha'] = df['Return'] - df['Return_Market']

In [ ]:
# Set index as id and sort the values by date then name
df.set_index('ID', inplace=True, drop=False)
df = df.sort_values(['Date','Name'], ascending=(True,True)).reset_index(drop=True)

In [ ]:
# Get rolling mean return and volatility of returns
grouped = df.groupby('Name')
df['Mean_Return'] = grouped['Return'].rolling(window=12, min_periods=4).mean().reset_index(0, drop=True)
df['Volatility'] = grouped['Return'].rolling(window=12, min_periods=4).std().reset_index(0, drop=True)

In [ ]:
# sort values by name then date
df = df.sort_values(['Name','Date'], ascending=(True,True))

### Risk-free Rate

In [ ]:
# Read EM data to obtain risk-free rate
rf = pd.read_csv('EM_factors.csv', index_col='Date')

In [ ]:
rf.index = pd.to_datetime(rf.index, format='%Y%m')
rf['RF'] = rf['RF'] / 100  # Convert RF to decimal 

In [ ]:
df1 = df

In [ ]:
# Calculate the quarterly risk-free rate
rf['RF'] = rf['RF'] + 1
quarterly_rf = rf['RF'].resample('Q').apply(lambda x: x.prod() - 1).reset_index()
quarterly_rf.columns = ['Date', 'RF']

# Ensure the Date column is normalized to remove any time component
quarterly_rf['Date'] = quarterly_rf['Date'].dt.normalize()

df1['Date'] = pd.to_datetime(df1['Date'])

# Create a mapping of the closest quarterly dates to the main dataset dates
unique_dates = sorted(df1['Date'].unique())
date_mapping = {}
for date in unique_dates:
    closest_date = min(quarterly_rf['Date'], key=lambda x: abs(x - date))
    date_mapping[date] = closest_date

# Add a new column to the main dataset for the mapped dates
df1['Mapped_Date'] = df1['Date'].map(date_mapping)

# Merge the quarterly risk-free rates with the main dataset
merged_df = pd.merge(df1, quarterly_rf, left_on='Mapped_Date', right_on='Date', how='left')

# Drop the temporary 'Mapped_Date' and duplicate 'Date' columns
merged_df.drop(columns=['Mapped_Date', 'Date_y'], inplace=True)
merged_df.rename(columns={'Date_x': 'Date'}, inplace=True)

In [ ]:
df = merged_df

In [ ]:
# Calculate excess returns
df['Excess Return'] = df['Return'] - df['RF']

In [ ]:
columns = [
    'Date', 'Name', 'ID', 'Currency', 'Price', 'Total Assets', 'Accounts Payable', 
    'Total Liabilities', 'Net Income', 'EBIT', 'Revenue', 'Depreciation', 
    'Gross Profit', 'Inventories', 'Receivables', 'Sales/Turnover', 
    'Stockholders Equity', 'Interest Expense', 'EBITDA', 'Free Cash Flow', 
    'Financing Cash Flow', 'Investing Cash Flow', 'Operating Cash Flow', 
    'Current Liabilities', 'Current Assets', 'Cost of Goods Sold', 'ROA', 
    'ROE', 'ROI', 'Gross Profit Margin', 'Operating Profit Margin', 
    'Net Profit Margin', 'Current Ratio', 'Quick Ratio', 'D/E Ratio', 
    'Interest Coverage Ratio', 'Asset Turnover Ratio', 'Inventory Turnover Ratio', 
    'Operational Gearing', 'Revenue Growth', 'Earnings Growth', 'Asset Growth', 
    'Equity Growth', 'Accruals Ratio', 'Cashflow to Net Income Ratio', 'High Price', 
    'Low Price', 'Volume', 'Shares', 'Equity', 'Market Cap', 'Return', 'Return_Market',
    'Alpha', 'Mean_Return', 'Volatility', 'Excess Return', 'RF'
]

In [ ]:
df = df[columns]

## Shift Returns

In [ ]:
df2 = df

In [ ]:
df2['Date'] = pd.to_datetime(df2['Date'])

# Sort values by 'Name' and 'Date'
df2 = df2.sort_values(['Name', 'Date'], ascending=[True, True])

# Shift the 'Return' and 'Excess Return' columns one period backward for each 'Name'
df2['Return_adjusted'] = df2.groupby('Name')['Return'].shift(-1)
df2['RF_adjusted'] = df2.groupby('Name')['RF'].shift(-1)
df2['Excess_Return_adjusted'] = df2.groupby('Name')['Excess Return'].shift(-1)

In [ ]:
df = df2

## Finishing Touches

In [ ]:
df.index = df['Date']

In [ ]:
# Drop rows with NaN values in any of the specified columns
df = df.dropna(subset=columns)
df = df.dropna(subset=['Return_adjusted', 'RF_adjusted', 'Excess_Return_adjusted'])

In [ ]:
df = df[['Date', 'Name', 'ID', 'Currency', 'Price', 'Total Assets', 'Accounts Payable', 
    'Total Liabilities', 'Net Income', 'EBIT', 'Depreciation', 
    'Gross Profit', 'Inventories', 'Receivables', 'Sales/Turnover', 
    'Stockholders Equity', 'Interest Expense', 'EBITDA', 'Free Cash Flow', 
    'Financing Cash Flow', 'Investing Cash Flow', 'Operating Cash Flow', 
    'Current Liabilities', 'Current Assets', 'Cost of Goods Sold', 'ROA', 
    'ROE', 'ROI', 'Gross Profit Margin', 'Operating Profit Margin', 
    'Net Profit Margin', 'Current Ratio', 'Quick Ratio', 'D/E Ratio', 
    'Interest Coverage Ratio', 'Asset Turnover Ratio', 'Inventory Turnover Ratio', 
    'Operational Gearing', 'Revenue Growth', 'Earnings Growth', 'Asset Growth', 
    'Equity Growth', 'Accruals Ratio', 'Cashflow to Net Income Ratio', 'High Price', 
    'Low Price', 'Volume', 'Shares', 'Equity', 'Market Cap',
    'Alpha', 'Mean_Return', 'Volatility', 'Excess_Return_adjusted', 'RF_adjusted', 'Return_adjusted']]

In [ ]:
# Drop rows where name is duplicated on the same date and keep the first
df = df.drop_duplicates(['Name','Date'],keep= 'first')

In [ ]:
# Reset index of main data
df = df.reset_index(drop=True)

In [ ]:
# Sort values by name and then date
df = df.sort_values(['Name','Date'], ascending=(True,True)).reset_index(drop=True)

In [ ]:
# Drop rows with +-inf
specified_columns = ['Interest Coverage Ratio', 'Inventory Turnover Ratio', 
                     'Revenue Growth', 'Earnings Growth', 'Cashflow to Net Income Ratio']

for col in specified_columns:
    df = df[~np.isinf(df3[col])]

In [ ]:
df.to_csv("EM_universe.csv", index = False)